## Gurobi Installation

If you are using Gurobi as an academic you should do the following for installation:

1. Register for a free Gurobi account as an academic and log in.
2. Visit the Download Gurobi Optimizer page and download the version you need, as well as the README.txt.
3. Follow the instructions in README.txt to install the software.
4. Once installed, visit the Gurobi User Portal to request your free **Named-User License**.
5. Next, run grbgetkey using the argument provided on the Academic License Detail page (ex: grbgetkey ae36ac20-16e6-acd2-f242-4da6e765fa0a). The grbgetkey program will prompt you to store the license file on your machine.

[Gurobi's Youtube channel](https://www.youtube.com/@GurobiVideos/playlists) has tutorials for installation among other useful content to undestand Gurobi's software and optimization.

In [12]:
#%pip install gurobipy

In [13]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import gurobipy as gp
from gurobipy import GRB
import matplotlib.pyplot as plt

## Orange juice data

The data used is the Store-Level Panel Data on Orange Juice Sales from Bayesm library in R, which contains weekly sales of refrigerated orange juice at 83 stores. We will focus on store 2.

The data provides the logarithm of demand, the price, the status of deal (coupon activity) and feature advertisement for each product each week. 

In [14]:
data=pd.read_csv("Orangejuice.csv")
data_store2=data[data["store"]== 2]

nprod=11 
n=list()#list with index of products
for i in range(1, nprod+1):
    n.append(i)

log_move = data_store2.pivot(index='week', columns='brand', values='logmove')# to get log move for every product each week
log_move.reset_index(inplace=True)
log_move=log_move[n] 

deal = data_store2.pivot(index='week', columns='brand', values='deal')
deal.reset_index(inplace=True)
deal=deal[n] 

feat = data_store2.pivot(index='week', columns='brand', values='feat')
feat.reset_index(inplace=True)
feat=feat[n] 

col_log_move=[] #column names for Logdemand for all the products
col_prices=[]#column names for price for all the products
col_logprices =[] #for Logprice
col_deal=[] #for deal (coupon activity)
col_feat=[] #for feat (feature advertisement)
for i in n:
    col_log_move.append(f"Logdemand{i}")
    col_prices.append(f"price{i}")
    col_logprices.append(f"Logprice{i}")
    col_deal.append(f"Deal{i}")
    col_feat.append(f"Feat{i}")

log_move.columns=col_log_move

deal.columns=col_deal
feat.columns=col_feat

Prices=data_store2[col_prices] 
log_price=np.log(Prices)
log_price.columns=col_logprices

demanda= pd.concat([log_move, log_price, deal, feat], axis=1)
demanda=pd.DataFrame(demanda.dropna())

Dataframe demanda has the log demanda, log prices, deal and feat for the 11 products each week, being each row a week 

### Linear Regression

With linear regression we will predict the demand given the price, deal and feat of each product, considering the cross elasticity between the products. The 11 regressions are stored in a dictonary for easy access.

In [15]:
model_dict = {}
price_term = ' + '.join(col_logprices+col_deal+col_feat) #to make the formula easier to write

for i in range(1, nprod + 1):
    formula = f"demanda['Logdemand{i}'] ~ {price_term}"
    modelo = smf.ols(formula, data=demanda).fit()
    model_dict[f'm_{i}'] = modelo

## Optimization model

We want to decide the price of 11 products to maximize the Category Revenue of orange juice in a superstore. 

#### Variables:

$p_i$: Price of product $i$

$q_i$: Predicted demand for product $i$ considering prices and elasticities between products.

#### Constraints:
- $preciomin_i$: minimun historical price of product $i$ 

- $preciomax_i$: maximun historical price of product $i$

- Average price of category can change up to 10%:
\begin{align}
\textrm{mean(Prices)} * 0.9 \leq \textrm{mean}\left(\sum_{i} p_i\right) \leq \textrm{mean(Prices)} * 1.1 \nonumber
\end{align}

#### Objective Function:

\begin{align}
\textrm {maximize} &  \sum_{i}  (p_i * q_i)& \nonumber
\end{align}

## Gurobi

To optimize with gurobi we have to add our variables and constraints to an already created model. \
Firts, we define the min, max and mean of the prices to be able to make our constraints easily.


In [16]:
preciomin = np.nanmin(Prices, axis=0)
preciomax=np.nanmax(Prices, axis=0)

precio_min={i: preciomin[i-1] for i in n} 
precio_max={i: preciomax[i-1] for i in n} 
log_min={i: np.log(precio_min[i]) for i in n}
log_max={i: np.log(precio_max[i]) for i in n}

prom=[np.mean(np.mean(Prices,axis=1))*nprod*0.9, np.mean(np.mean(Prices, axis=0))*nprod*1.1] 

Now we can start creating the gurobi model.
- gp.Model: creates model

- model.addVars: adds variable to the model

- model.addConstr: adds contraint to the model

- model.addGenConstrExp(a, b): makes a general constraint for exp(a)=b

- model.setObjective: defines the objective function

- model.ModelSense: defines if we are maximizing or minimizing

- Params.NonConvex: set to 2 means specifies that the objective function is non convex and solves it by translating it into bilinear form and applying spatial branching.

- model.optimize: optimizes the model 

In [17]:
m= gp.Model("Orange Juice Prices with 11 products")
p= m.addVars(n, name="p", lb=precio_min, ub=precio_max) #variable for price
log_p=m.addVars(n, name="log_p", lb= log_min, ub=log_max) #variable for log_price
log_q = m.addVars(n, name="log_q", lb=0) #variable for log_demand
q = m.addVars(n, name="q", lb=0)  #variable for demand

for i in n:
    m.addGenConstrExp(log_p[i], p[i]) #add constraint for consistency between log(p) and p
    m.addGenConstrExp(log_q[i], q[i])

#contraint: average price of category 
m.addConstr(sum((p[i] for i in n)) >= prom[0]) 
m.addConstr(sum((p[i] for i in n)) <= prom[1])

#to predict log(demand) we use the coefficients of each linear regression previously done
for i in n:
    coef=model_dict[f"m_{i}"].params.to_dict()
    pred=coef["Intercept"]+ sum((coef[f"Logprice{i}"]*log_p[i] for i in n)) #we are assuming  that deal=0 and feast=0
    m.addConstr(log_q[i]==pred) 

m.setObjective(sum((p[i]*q[i] for i in n)))
m.ModelSense = GRB.MAXIMIZE

m.Params.NonConvex = 2
m.optimize()

Restricted license - for non-production use only - expires 2024-10-28
Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 13 rows, 44 columns and 154 nonzeros
Model fingerprint: 0x54b1ea71
Model has 11 quadratic objective terms
Model has 22 general constraints
Variable types: 44 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-03, 4e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [2e-02, 4e+00]
  RHS range        [4e-01, 9e+00]
Presolve added 55 rows and 3220 columns
Presolve time: 0.04s
Presolved: 91 rows, 3276 columns, 9906 nonzeros
Presolved model has 22 SOS constraint(s)
Presolved model has 11 bilinear constraint(s)
Variable types: 3276 continuous, 0 integer (0 binary)

Root relaxation: 

     0     0 29190.7809    0   21          - 29190.7809      -     -    0s
     0     2 29190.7809    0   21          - 29190.7809      -     -    0s
H  538   388                    3222.8822149 27780.2243   762%   5.6    1s
H 1109   739                    3302.1854193 27780.2243   741%   5.2    1s
H 1246   850                    3302.7577231 27780.2243   741%   5.1    1s
H 1256   834                    3419.8696232 27780.2243   712%   5.1    1s
* 1356   867             164    3419.9131895 27780.2243   712%   5.0    1s
H 1526   908                    3420.8477180 27780.2243   712%   5.1    1s
H 2479  1117                    3522.3959443 18620.6324   429%   8.7    4s
H 2490   842                    6479.0426608 18620.6324   187%   8.7    4s
H 2510   759                    6673.9968869 18620.6324   179%   8.7    4s
H 2901   808                    6683.5857468 16039.9900   140%   8.3    4s
  3118   853 6997.43919   21  122 6683.58575 13823.1695   107%   8.3    5s
H 3999  1008             

Since this optimization is non convex, Gurobi does Branch and Cut algorithm.

To undestand the output of this optimization:
- Nodes:
	- Expl: number of branch and cut nodes that have been explored.
	- Unexpl: number of leaf nodes in the search that remeain unexplored.
	- H->new feasible solution has been found with MIP Heuristic.
	- *->new feasible solution has been found by branching.
- Current Node:
	- Obj: objective of tha associated relaxation.
	- Depth: depth of the node in the branch and cut tree.
	- Intinf: num of integer variables that have non intefral values in the associated relaxation.
- Objective bounds: 
	- Incumbent: objective value of the current incumbent.
	- Bestbd: current objective bound provided by leaf nodes of the search tree.
	- Gap: realtive gap between the two objectives, incumbent and best bound.
	
		The optimal objective value is always between the incumbent and bestbd.
- Work:
	- It/node: average number of simplex iterations performed per node to that point.
- Time: elapsed time since de solve began.

[For more information.](https://www.gurobi.com/documentation/9.1/refman/mip_logging.html) 


## Results

To visualize the result easily and be able to compare it with the prices, demand and category revenue before optimization, we make a function f_venta to calculate the income given the price of the products predicting the demand.

In [ ]:
def f_venta(Precio):
    
    lp = pd.DataFrame(np.array(Precio))
    lp.columns = col_logprices + col_deal + col_feat
    lp[col_logprices]=np.log(lp[col_logprices])
    venta = 0
    for i in range(1, nprod + 1):
        v = Precio.iloc[0,i-1] * np.round(np.exp(model_dict[f'm_{i}'].predict(lp)))
        venta = venta + v
    return np.array(venta)[0]

We will compare the results of the optimization with the last week's prices 

In [22]:
solution = pd.DataFrame()
#preparing to compare the optimization with the last week of data
lastdemand=np.array(np.exp(log_move))[-1,:]
Preciolast=np.array(Prices)[-1,:]
Preciolast_df=pd.DataFrame(Preciolast).T
Preciolast_df[col_deal]=0
Preciolast_df[col_feat]=0

opt_revenue = m.ObjVal#extraccting the z value from the optimization

solution['Product'] = n
solution['Last Price'] = Preciolast
solution['Last demand']=lastdemand
solution['Last Sales']=lastdemand*Preciolast
solution['Price opt'] = [(p[i].X) for i in n] #extracting the values of variable p from the model
solution['Demand opt'] = [(q[i].X) for i in n]
solution['Sales opt'] = [(q[i].X * p[i].X) for i in n]
solution['Change in Price'] =[f"{(round((p[i].X-Preciolast[i-1])*100/Preciolast[i-1],1))}%" for i in n]

print(f"Income after optimization: {opt_revenue}")
print(f"Income without optimization: {f_venta(Preciolast_df)}") #predicting demand with the last prices, considering deal and feal equal to 0
print(f"% Increase of income after optimization: {round((opt_revenue-f_venta(Preciolast_df))/f_venta(Preciolast_df)*100,2)}%")

solution=solution.round(3)
display(solution)

Income after optimization: 7646.826775338873
Income without optimization: 2283.2134969237004
% Increase of income after optimization: 234.92%


,Product,Last Price,Last demand,Last Sales,Price opt,Demand opt,Sales opt,Change in Price
0,1,0.046,5824.0,270.270,0.026,14807.988,391.023,0.569
1,2,0.042,8256.0,343.140,0.037,15730.481,588.254,0.900
2,3,0.047,1280.0,59.800,0.052,749.500,39.232,1.120
3,4,0.043,1984.0,85.560,0.048,2352.177,112.463,1.109
4,5,0.034,41216.0,1410.360,0.050,3887.350,192.545,1.447
5,6,0.037,1824.0,67.323,0.053,658.115,34.894,1.436
6,7,0.040,384.0,15.540,0.018,22996.766,406.037,0.436
7,8,0.032,1792.0,57.960,0.018,3132.304,55.305,0.546
8,9,0.037,704.0,26.290,0.048,46.878,2.256,1.289
9,10,0.028,8640.0,245.700,0.015,366776.096,5673.568,0.544
